In [1]:
import re
import pandas as pd

In [2]:
def editDistance(word1, word2):
    len1 = len(word1)
    len2 = len(word2)
    distanceMap = [[0] for i in range(len1+1)]
    distanceMap[0] = [i for i in range(len2+1)]
    for i in range(len1+1):
        distanceMap[i][0] = i
    
    for i in range(len1):
        for j in range(len2):
            adder = 0 if(word1[i]==word2[j]) else 1
            dist = min(distanceMap[i+1][j]+1,distanceMap[i][j+1]+1,distanceMap[i][j]+adder)
            distanceMap[i+1].append(dist)
    
    return distanceMap[len1][len2]

In [3]:
def D_L_editDistance(word1, word2):
    len1 = len(word1)
    len2 = len(word2)
    distanceMap = [[0] for i in range(len1+1)]
    distanceMap[0] = [i for i in range(len2+1)]
    for i in range(len1+1):
        distanceMap[i][0] = i
    
    for i in range(1,len1+1):
        for j in range(1,len2+1):
            adder = 0 if(word1[i-1]==word2[j-1]) else 1
            #print(i,j,adder)
            dist = min(distanceMap[i][j-1]+1,distanceMap[i-1][j]+1,distanceMap[i-1][j-1]+adder)
            if(i>1 and j>1 and word1[i-2:i] == word2[j-2:j][::-1]):
                dist = min(distanceMap[i-2][j-2]+1, dist)
            distanceMap[i].append(dist)
    #print(distanceMap)
    return distanceMap[len1][len2]

In [32]:
def D_L_Backtrack(word1, word2):
    len1 = len(word1)
    len2 = len(word2)
    if(word1==''):
        return [('insertion', word2, '@')]
    elif(word2 == ''):
        return [('deletion', word1, '@')]
    distanceMap = [[0] for i in range(len1+1)]
    distanceMap[0] = [i for i in range(len2+1)]
    for i in range(len1+1):
        distanceMap[i][0] = i
    
    for i in range(1,len1+1):
        for j in range(1,len2+1):
            adder = 0 if(word1[i-1]==word2[j-1]) else 1
            #print(i,j,adder)
            dist = min(distanceMap[i][j-1]+1,distanceMap[i-1][j]+1,distanceMap[i-1][j-1]+adder)
            if(i>1 and j>1 and word1[i-2:i] == word2[j-2:j][::-1]):
                dist = min(distanceMap[i-2][j-2]+1, dist)
            distanceMap[i].append(dist)
            
    i = len1
    j = len2
    operations = []
    while(i>=1 and j>=1):
        if(distanceMap[i][j] == distanceMap[i][j-1]+1):
            #print('insertion of %s after %s' % (word2[j-1], word1[i-1]))
            operations.append(('insertion', word2[j-1], word1[i-1]))
            j=j-1
        elif(distanceMap[i][j] == distanceMap[i-1][j]+1):
            #print('deletion of %s after %s' % (word1[i-1], word1[i-2]))
            operations.append(('deletion', word1[i-1], word1[i-2]))
            i=i-1
        elif(distanceMap[i][j] == distanceMap[i-1][j-1] + (0 if word1[i-1] == word2[j-1] else 1)):
            if(not word1[i-1] == word2[j-1]):    
                #print('substitution of %s with %s' % (word2[j-1], word1[i-1]))
                operations.append(('substitution', word2[j-1], word1[i-1]))
            i=i-1
            j=j-1
        else:
            #print('transposition of %s with %s' % (word2[j-1], word1[i-1]))
            operations.append(('transposition', word2[j-1], word1[i-1]))
            i=i-2
            j=j-2
    word1=word1[:-1]+'@'
    word2=word2[:-1]+'@'
    while(i>0):
        #print('deletion of %s after %s' % (word1[i-1], word1[i-2]))
        operations.append(('deletion', word1[i-1], word1[i-2]))
        i=i-1
    while(j>0):
        #print('insertion of %s after %s' % (word2[j-1], word1[i-1]))
        operations.append(('insertion', word2[j-1], word1[i-1]))
        j=j-1
    return operations

In [5]:
def tokenMe(inputString):
    inputString = inputString.lower()
    outputString = ''
    for ch in inputString.replace('--',' ').replace('\'', ''):
        if(ch.isalpha() or ch == ' ' or ch.isnumeric()):
            outputString+=ch
        else:
            outputString+=' '
    return outputString

In [6]:
corpus = open('corpusMini.txt').read().lower()
corpus2 = tokenMe(corpus)

In [7]:
test_words_correct = []
with open('test-words-correct.txt') as fp:
    line = fp.readline()
    while(line):
        line = tokenMe(line)
        test_words_correct.append(line[:-1])
        line = fp.readline()
test_words_misspelled = []
with open('test-words-misspelled.txt') as fp2:
    line = fp2.readline()
    while(line):
        line = tokenMe(line)
        test_words_misspelled.append(line[:-1])
        line = fp2.readline()

In [8]:
spell_errors = []
with open('spell-errors.txt') as fp:
    line = fp.readline()
    while(line):
        spell_errors.append(line[:-1])
        line = fp.readline()

In [9]:
spell_error_samples = {}
for spl_error in spell_errors:
    spl = spl_error.split(':')
    spell_error_samples[spl[0].lower()] = []
    splErrors = spl[1].split(',')
    for err in splErrors:
        spell_error_samples[spl[0].lower()].append(err.replace(' ', '').lower())
for se in spell_error_samples.items():
    spell_error_samples[se[0]] = []
    for err in se[1]:
        if('*' in err):
            sp = err.split('*')
            spell_error_samples[se[0]].append((sp[0], int(sp[1])))
        else:
            spell_error_samples[se[0]].append((err,1))

In [12]:
def char_position(letter):
    return ord(letter) - 97
letters    = 'abcdefghijklmnopqrstuvwxyz@*'

def createConfusionMatricesFromCorrections():
    confusion_matrix_substitution = [[0 for i in range(len(letters))] for j in range(len(letters))]
    confusion_matrix_transposition = [[0 for i in range(len(letters))] for j in range(len(letters))]
    confusion_matrix_insertion = [[0 for i in range(len(letters))] for j in range(len(letters))]
    confusion_matrix_deletion = [[0 for i in range(len(letters))] for j in range(len(letters))]
    confusion_matrix = {'deletion': confusion_matrix_deletion,
                       'insertion': confusion_matrix_insertion,
                       'substitution': confusion_matrix_substitution,
                       'transposition': confusion_matrix_transposition}
    for (key,spellErrors) in spell_error_samples.items():
        for spellError in spellErrors:
            corrections = D_L_Backtrack(spellError[0], key)
            for correction in corrections:
                    x = char_position(correction[1]) if 0<=char_position(correction[1])<=25 else 27
                    y = char_position(correction[2]) if correction[2] != '@' and 0<=char_position(correction[2])<=25 else 26 if correction[2] == '@' else 27
                    confusion_matrix[correction[0]][x][y]+=spellError[1]

    return confusion_matrix

In [13]:
confusion_matrix = createConfusionMatricesFromCorrections()
deletionFrame = pd.DataFrame(confusion_matrix['deletion'], columns = list('abcdefghijklmnopqrstuvwxyz@*'), index = list(letters))
insertionFrame= pd.DataFrame(confusion_matrix['insertion'], columns = list('abcdefghijklmnopqrstuvwxyz@*'), index = list(letters))
substitutionFrame=pd.DataFrame(confusion_matrix['substitution'], columns = list('abcdefghijklmnopqrstuvwxyz@*'), index = list(letters))
transpositionFrame=pd.DataFrame(confusion_matrix['transposition'], columns = list('abcdefghijklmnopqrstuvwxyz@*'), index = list(letters))

In [23]:
print(insertionFrame);

     a    b    c    d    e    f    g    h    i   j ...    s     t    u    v  \
a   38   58   98   69  644   37   69   59  294   1 ...  101   237  174   21   
b   31   20    0   14   14    3    4    1    5   0 ...    3    18   21    2   
c  180    8  208   12  183   42   17   18  139   0 ...  358    85   43    7   
d   73   20    7   67  398    3   32   60   46   1 ...   56   245   26    7   
e  321  110  139  219  280   87  195  223  354   6 ...  513  1078  166  128   
f   16    4    7    2   32  137    2    6   17   0 ...    7    12    2   16   
g   55   10   13   47   79    5   81    9  121   7 ...   18    30   29    3   
h   30    2  312   16   61  148   57    1   67   1 ...  129   111   28   18   
i  481   32  256   88  487   96   61   73  141   1 ...  337   291  203   35   
j    0    0    0    0   17    0    4    0    0   0 ...    0     0    0    0   
k    7    0   54    2    1    0    8    1    1   0 ...   17     4    2    0   
l  102   31   34   35  213   32   24   39   81   0 .

In [14]:
word_list=re.split('\s+',corpus2)
word_list_set = set(word_list)

In [15]:
numTokens = len(word_list)
print("Number of tokens: %d" %numTokens)

Number of tokens: 3383


In [16]:
wordFreqDf = pd.DataFrame({'word':[],'freq':[],'percentage':[]})
wordFreq = []
for word in word_list_set:
    count = word_list.count(word)
    wordFreq.append([word, count, round(count/numTokens,5)])
wordFreq = sorted(wordFreq, key=lambda tup: tup[2], reverse=True)
wordFreqDf = pd.DataFrame(wordFreq, columns=['word','count','percentage'])

## getWordFreq: Returns the frequency of occurrence of a word in the corpus

In [17]:
def getWordFreq(word):
    if(word not in word_list_set):
        return 0
    located = wordFreqDf.loc[wordFreqDf['word'] == word]
    return float(located['percentage'])

In [18]:
b = getWordFreq('the')
b

0.04789

In [75]:
def getProbabilityFromConfusionMatrix(operation):
    if(operation[0] == 'insertion'):
        series = insertionFrame.loc[operation[1]]
        return round(series[operation[2]]/sum(series), 7)
    elif(operation[0] == 'deletion'):
        series = deletionFrame.loc[operation[1]]
        return round(series[operation[2]]/sum(series), 7)

    elif(operation[0] == 'substitution'):
        series = substitutionFrame.loc[operation[1]]
        return round(series[operation[2]]/sum(series), 7)

        return substitutionFrame.loc[operation[1]][operation[2]]
    elif(operation[0] == 'transposition'):
        series = transpositionFrame.loc[operation[1]]
        return round(series[operation[2]]/sum(series), 7)

## getCandidates: returns the candidate spell checkings of a word ( edit distance is 1 ) 

In [72]:
def getCandidates(word):
    candidates_len1 = [word for word in word_list_set if(abs(len(word)-len(testWord))<=1)]
    candidates_edit1 = [candidate for candidate in candidates_len1 if(editDistance(candidate, testWord)==1)]
    return candidates_edit1

In [73]:
testWord = 'nd'
cands = getCandidates(testWord)
cands

['end', 'and', 'no']

In [ ]:
print('test word: ', testWord)
print()
best = 0
for cand in cands:
    print('cand: ', cand)
    px = getWordFreq(cand)
    print('freq\t\t: ', px)
    operation = D_L_Backtrack(testWord, cand)
    print('operation: ', operation)
    pcm = getProbabilityFromConfusionMatrix(operation[0])
    print('probability confusion matrix: ', pcm)
    print('multiplied: ', pcm*px)
    if(pcm*px>best):
        best = pcm*px
    print(best)

In [44]:
operation

[('insertion', 'e', '@')]

0.0056154

In [69]:
print([getWordFreq(cand) for cand in cands])

[0.0003, 0.02838, 0.00236]


In [70]:
D_L_Backtrack(cands[0], testWord)

[('deletion', 'e', '@')]

In [35]:
D_L_Backtrack('ab', '')

[('deletion', 'ab', '@')]

In [ ]:
D_L_editDistance('oslo', 'snow')

In [ ]:
D_L_Backtrack('oslo', 'snow')